# Midterm 2

## FINM 36700 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is 100 points.
* You have 120 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.


Submission
* You will upload your solution to the `Midterm 2` assignment on Canvas, where you downloaded this. 
* Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers.)
* Your submission should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo - you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_2_data.xlsx`

This file contains the following sheets:
- for Section 2:
    * `sector stocks excess returns` - MONTHLY excess returns for 49 sector stocks
    * `factors excess returns` - MONTHLY excess returns of AQR factor model from Homework 5
- for Section 3:
    * `factors excess returns` - MONTHLY excess returns of AQR factor model from Homework 5

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 25     |
| 2       | 40     |
| 3       | 35     |

### Each numbered question is worth 5 points unless otherwise specified.

# 1. Short Answer

#### No Data Needed

These problems do not require any data file. Rather, analyze them conceptually. 

### 1.1.

Historically, which pricing factor among the ones we studied has shown a considerable decrease in importance?

The SMB/Size factor has shown a decrease in importance. This is due to markets becoming more efficient, greater investor awareness, and structural changes to the market where small cap stocks are now more available.

### 1.2.

True or False: For a given factor model and a set of test assets, the addition of one more factor to that model will surely decrease the cross-sectional MAE. 

True or False: For a given factor model and a set of test assets, the addition of one more factor to that model will surely decrease the time-series MAE. 

Along with stating T/F, explain your reasoning for the two statements.

False. Adding an additional factor can introduce noise if the factor isnt capturing significant cross section variation in asset returns, and is not guarenteed to decrease CS MAE

False. A new factor could be irrelevant, introduce multicollinearity, and/or overfit the model, which would not decrease TS MAE


### 1.3.

Consider the scenario in which you are helping two people with investments.

* The young person has a 50 year investment horizon.
* The elderly person has a 10 year investment horizon.
* Both individuals have the same portfolio holdings.

State who has the more certain cumulative return and explain your reasoning.

The elderly person with a 10 year horizon. Over a short time period, the effects of continious compounding is limited, and the cumulative return will be more predictable.

### 1.4.

Suppose we find that the 10-year bond yield works well as a new pricing factor, along with `MKT`.

Consider two ways of building this new factor.
1. Directly use the index of 10-year yields, `YLD`
1. Construct a Fama-French style portfolio of equities, `FFYLD`. (Rank all the stocks by their correlation to bond yield changes, and go long the highest ranked and shor tthe lowest ranked.)

Could you test the model with `YLD` and the model with `FFYLD` in the exact same ways? Explain.

No, we would not test these the same way. YLD is a direct measure of bond yields and should be tested as an individual variable in regressions, while FFYLD is a portfolio based factor and needs to be included/tested similar to how we test other factors in the Fama French model.

### 1.5.

Suppose we implement a momentum strategy on cryptocurrencies rather than US stocks.

Conceptually speaking, but specific to the context of our course discussion, how would the risk profile differ from the momentum strategy of US equities?

Cryptocurrencies incur much higher volatility, price swings, and less established momentum premiums. This would lead to a far less stable momentum factor. This strategy would also be suseptcible to other issues like low liquidity, larger tail risk, etc.

***

# 2. Pricing and Tangency Portfolio

You work in a hedge fund that believes that the AQR 4-Factor Model (present in Homework 5) is the perfect pricing model for stocks.

$$
\mathbb{E} \left[ \tilde{r}^i \right] = \beta^{i,\text{MKT}} \mathbb{E} \left[ \tilde{f}_{\text{MKT}} \right] + \beta^{i,\text{HML}} \mathbb{E} \left[ \tilde{f}_{\text{HML}} \right] + \beta^{i,\text{RMW}} \mathbb{E} \left[ \tilde{f}_{\text{RMW}} \right] + \beta^{i,\text{UMD}} \mathbb{E} \left[ \tilde{f}_{\text{UMD}} \right]
$$

The factors are available in the sheet `factors excess returns`.

The hedge fund invests in sector-tracking ETFs available in the sheet `sectors excess returns`. You are to allocate into these sectors according to a mean-variance optimization with...

* regularization: elements outside the diagonal covariance matrix divided by 2.
* modeled risk premia: expected excess returns given by the factor model rather than just using the historic sample averages.

You are to train the portfolio and test out-of-sample. The timeframes should be:
* Training timeframe: Jan-2018 to Dec-2022.
* Testing timeframe: Jan-2023 to most recent observation.

### 2.1.
(8pts)

Calculate the model-implied expected excess returns of every asset.

The time-series estimations should...
* NOT include an intercept. (You assume the model holds perfectly.)
* use data from the `training` timeframe.

With the time-series estimates, use the `training` timeframe's sample average of the factors as the factor premia. Together, this will give you the model-implied risk premia, which we label as
$$
\lambda_i := \mathbb{E}[\tilde{r}_i]
$$

* Store $\lambda_i$ and $\boldsymbol{\beta}^i$ for each asset.
* Print $\lambda_i$ for `Agric`, `Food`, `Soda`

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

factors_rts = pd.read_excel("midterm_2_data.xlsx", sheet_name='factors excess returns').set_index('date')
sector_rts = pd.read_excel("midterm_2_data.xlsx", sheet_name='sector excess returns').set_index('date')

start = '2018-01-01'
end = '2022-12-31'

factors_train = factors_rts.loc[start:end]
sector_train = sector_rts.loc[start:end]

factor_names = ['MKT', 'HML', 'RMW', 'UMD']
factor_premia = factors_train.mean()

betas = pd.DataFrame(index=sector_train.columns, columns=factor_names)
lambdas = pd.Series(index=sector_train.columns, dtype=np.float64)

for asset in sector_train.columns:
    y = sector_train[asset].values
    X = factors_train.values
    model = LinearRegression(fit_intercept=False)
    model.fit(X, y)
    beta_i = model.coef_
    betas.loc[asset] = beta_i
    lambdas[asset] = np.dot(beta_i, factor_premia.values)

print(lambdas[:3])

Agric    0.003655
Food     0.005454
Soda     0.007336
dtype: float64


### 2.2.

Use the expected excess returns derived from (2.1) with the **regularized** covariance matrix to calculate the weights of the tangency portfolio.

- Use the covariance matrix only for `training` timeframe.
- Calculate and store the vector of weights for all the assets.
- Return the weights of the tangency portfolio for `Agric`, `Food`, `Soda`.

$$
\textbf{w}_{t} = \dfrac{\tilde{\Sigma}^{-1} \bm{\lambda}}{\bm{1}' \tilde{\Sigma}^{-1} \bm{\lambda}}
$$

Where $\tilde{\Sigma}^{-1}$ is the regularized covariance-matrix.

In [2]:
cov_matrix = sector_train.cov()
cov_matrix_reg = cov_matrix.copy()

#chat GPT prompt: help me create a regularzied covariance matrix

for i in cov_matrix_reg.columns:
    for j in cov_matrix_reg.index:
        if i != j:
            cov_matrix_reg.loc[i, j] /= 2

inv_cov_matrix = np.linalg.inv(cov_matrix_reg)

#chat GPT: how to get tangency portfolio

onevec = np.ones(len(lambdas))
numerator = inv_cov_matrix.dot(lambdas.values)
denominator = onevec.T.dot(numerator)
wghts = numerator / denominator

weights_df = pd.Series(wghts, index=sector_train.columns)

print(weights_df[:3])

Agric   -0.030723
Food     0.015320
Soda     0.132944
dtype: float64


### 2.3.

Evaluate the performance of this allocation in the `testing` period. Report the **annualized**
- mean
- vol
- Sharpe

In [3]:
start_test = '2023-01-01'
sector_test = sector_rts.loc[start_test:]

sector_test = sector_test[weights_df.index]
portfolio_returns_test = sector_test.dot(weights_df)
mean_rt = portfolio_returns_test.mean()
vol_rt = portfolio_returns_test.std()

mean_rt_ann = mean_rt * 12
vol_rt_ann = vol_rt * np.sqrt(12)
sharpe_ratio = mean_rt_ann / vol_rt_ann


#chat GPT: create print statements for these variables

print(f"Annualized Mean Return: {mean_rt_ann:.4f}")
print(f"Annualized Volatility: {vol_rt_ann:.4f}")
print(f"Annualized Sharpe Ratio: {sharpe_ratio:.4f}")


Annualized Mean Return: 0.1812
Annualized Volatility: 0.1195
Annualized Sharpe Ratio: 1.5155


### 2.4.

(7pts)

Construct the same tangency portfolio as in `2.2` but with one change:
* replace the risk premia of the assets, (denoted $\lambda_i$) with the sample averages of the excess returns from the `training` set.

So instead of using $\lambda_i$ suggested by the factor model (as in `2.1-2.3`) you're using sample averages for $\lambda_i$.

- Return the weights of the tangency portfolio for `Agric`, `Food`, `Soda`.

Evaluate the performance of this allocation in the `testing` period. Report the **annualized**
- mean
- vol
- Sharpe

In [4]:
lambda_avg = sector_train.mean()
num_avg = inv_cov_matrix.dot(lambda_avg.values)
denom_avg = onevec.T.dot(num_avg)
wghts_sample = num_avg / denom_avg
weights_df_sample = pd.Series(wghts_sample, index=sector_train.columns)

#Chat gpt: print these

print("Tangency Portfolio Weights (Using Sample Averages) for Agric, Food, Soda:")
print(weights_df_sample[:3])

sector_test = sector_test[weights_df_sample.index]

portfolio_returns_test_sample = sector_test.dot(weights_df_sample)
mean_rt_sample = portfolio_returns_test_sample.mean()
vol_rt_sample = portfolio_returns_test_sample.std()

mean_rt_ann_sample = mean_rt_sample * 12
vol_rt_ann_sample = vol_rt_sample * np.sqrt(12)
sharpe_ratio_sample = mean_rt_ann_sample / vol_rt_ann_sample

#chat gpt: print these

print("Tangency Portfolio Performance (Using Sample Averages) in Testing Period:")
print(f"Annualized Mean Return: {mean_rt_ann_sample:.4f}")
print(f"Annualized Volatility: {vol_rt_ann_sample:.4f}")
print(f"Annualized Sharpe Ratio: {sharpe_ratio_sample:.4f}")


Tangency Portfolio Weights (Using Sample Averages) for Agric, Food, Soda:
Agric    0.14409
Food    -0.06981
Soda     0.32268
dtype: float64
Tangency Portfolio Performance (Using Sample Averages) in Testing Period:
Annualized Mean Return: 0.1768
Annualized Volatility: 0.1530
Annualized Sharpe Ratio: 1.1555


### 2.5.

Which allocation performed better in the `testing` period: the allocation based on premia from the factor model or from the sample averages?

Why might this be?

The allocation based on premia from the factor model performs better, as it has a Sharpe Ratio of 1.5155 vs 1.1555. This may be because factor models are less prone to overfitting and account for systematic risk.

### 2.6.
Suppose you now want to build a tangency portfolio solely from the factors, without using the sector ETFs.

- Calculate the weights of the tangency portfolio using `training` data for the factors.
- Again, regularize the covariance matrix of factor returns by dividing off-diagonal elements by 2.

Report, in the `testing` period, the factor-based tangency stats **annualized**...
- mean
- vol
- Sharpe


In [5]:
cov_factors = factors_train.cov()
cov_factors_reg = cov_factors.copy()
for i in cov_factors_reg.columns:
    for j in cov_factors_reg.index:
        if i != j:
            cov_factors_reg.loc[i, j] /= 2
inv_cov_factors = np.linalg.inv(cov_factors_reg)

#chat gpt: how to find premias

num_factors = inv_cov_factors.dot(factor_premia.values)
denominator_factors = np.ones(len(factor_premia)).T.dot(num_factors)
wghts_factors = num_factors / denominator_factors
weights_factors_df = pd.Series(wghts_factors, index=factor_names)

factors_test = factors_rts.loc[start_test:]
factor_returns_test = factors_test
portfolio_returns_test_factors = factor_returns_test.dot(weights_factors_df)

mean_rt_factors = portfolio_returns_test_factors.mean()
vol_rt_factors = portfolio_returns_test_factors.std()
mean_rt_ann_factors = mean_rt_factors * 12
vol_rt_ann_factors = vol_rt_factors * np.sqrt(12)
sharpe_ratio_factors = mean_rt_ann_factors / vol_rt_ann_factors

#chat gpt: print these variables

print("Factor-Based Tangency Portfolio Performance in Testing Period:")
print(f"Annualized Mean Return: {mean_rt_ann_factors:.4f}")
print(f"Annualized Volatility: {vol_rt_ann_factors:.4f}")
print(f"Annualized Sharpe Ratio: {sharpe_ratio_factors:.4f}")


Factor-Based Tangency Portfolio Performance in Testing Period:
Annualized Mean Return: 0.0624
Annualized Volatility: 0.0582
Annualized Sharpe Ratio: 1.0719


### 2.7.

Based on the hedge fund's beliefs, would you prefer to use the ETF-based tangency or the factor-based tangency portfolio? Explain your reasoning. Note that you should answer based on broad principles and not on the particular estimation results.

I would prefer to use the factor based tangency portfolio. Investing in the factors closely aligns with the hedge fund's philosiphy, offers exposure to systematic risk premia without sector specific risks, and offers better risk adjuted returns by focusing on sources of expected return the model identifies.

***

# 3. Long-Run Returns

For this question, use only the sheet `factors excess returns`.

Suppose we want to measure the long run returns of various pricing factors.

### 3.1.

Turn the data into log returns.
- Display the first 5 rows of the data.

Using these log returns, report the **annualized**
* mean
* vol
* Sharpe

### 3.2.

Consider 15-year cumulative log excess returns. Following the assumptions and modeling of Lecture 6, report the following 15-year stats:
- mean
- vol
- Sharpe

How do they compare to the estimated stats (1-year horizon) in `3.1`? 

In [6]:
log_factor_rts = np.log(1 + factors_rts)
print(log_factor_rts.head())

mean_log_rts = log_factor_rts.mean()
vol_log_rts = log_factor_rts.std()

mean_log_rts_annualized = mean_log_rts * 12
vol_log_rts_annualized = vol_log_rts * np.sqrt(12)

sharpe_ratio_log = mean_log_rts_annualized / vol_log_rts_annualized

#chat gpt: put these in a dataframe

performance_df = pd.DataFrame({
    'Mean': mean_log_rts_annualized,
    'Volatility': vol_log_rts_annualized,
    'Sharpe': sharpe_ratio_log
})
print(performance_df)


                 MKT       HML       RMW       UMD
date                                              
1980-01-01  0.053636  0.017349 -0.017146  0.072786
1980-02-01 -0.012275  0.006081  0.000400  0.075849
1980-03-01 -0.138113 -0.010151  0.014494 -0.100373
1980-04-01  0.038932  0.010544 -0.021224 -0.004309
1980-05-01  0.051263  0.003793  0.003394 -0.011263
         Mean  Volatility    Sharpe
MKT  0.073549    0.158841  0.463033
HML  0.019770    0.109782  0.180081
RMW  0.043540    0.083573  0.520979
UMD  0.050095    0.160433  0.312248


In [7]:
T = 15

mean_cum_return_15y = mean_log_rts_annualized * T
vol_cum_return_15y = vol_log_rts_annualized * np.sqrt(T)
sharpe_ratio_15y = mean_cum_return_15y / vol_cum_return_15y

#chat gpt: put these in a data frame

performance_15y_df = pd.DataFrame({
    'Mean (15y)': mean_cum_return_15y,
    'Volatility (15y)': vol_cum_return_15y,
    'Sharpe (15y)': sharpe_ratio_15y
})
print(performance_15y_df)


     Mean (15y)  Volatility (15y)  Sharpe (15y)
MKT    1.103228          0.615188      1.793318
HML    0.296544          0.425182      0.697452
RMW    0.653096          0.323676      2.017743
UMD    0.751422          0.621353      1.209332


Over a longer time period, we get the benefits of time diversification, and our Sharpe ratios improve across the board.

### 3.3.

What is the probability that momentum factor has a negative mean excess return over the next 
* single period?
* 15 years?

In [8]:
#chat gpt: show me how to calculate factor performance probability

from scipy.stats import norm

mean_umd = mean_log_rts_annualized['UMD']
vol_umd = vol_log_rts_annualized['UMD']

z_single = (-mean_umd) / vol_umd
prob_negative_single = norm.cdf(z_single)

mean_cum_umd_15y = mean_umd * T
vol_cum_umd_15y = vol_umd * np.sqrt(T)
z_15y = (-mean_cum_umd_15y) / vol_cum_umd_15y
prob_negative_15y = norm.cdf(z_15y)

#chat gpt: print these

print(f"Probability that UMD has negative mean excess return over next year: {prob_negative_single:.4f}")
print(f"Probability that UMD has negative mean excess return over next 15 years: {prob_negative_15y:.4f}")

Probability that UMD has negative mean excess return over next year: 0.3774
Probability that UMD has negative mean excess return over next 15 years: 0.1133


### 3.4.

Recall from the case that momentum has been underperforming since 2009. 

Using data from 2009 to present, what is the probability that momentum *outperforms* the market factor over the next
* period?
* 15 years?

In [9]:
start = '2009-01-01'
data_2009 = factors_rts.loc[start:]
log_data = np.log(1 + data_2009 / 100)
log_excess = log_data['UMD'] - log_data['MKT']

mean_excess = log_excess.mean()
vol_excess = log_excess.std()

mean_annual = mean_excess * 12
vol_annual = vol_excess * np.sqrt(12)

z_single = -mean_annual / vol_annual
prob_under_single = norm.cdf(z_single)
prob_out_single = 1 - prob_under_single

mean_15y = mean_annual * T
vol_15y = vol_annual * np.sqrt(T)
z_15y = -mean_15y / vol_15y
prob_under_15y = norm.cdf(z_15y)
prob_out_15y = 1 - prob_under_15y

#chat gpt: print these results

print(f"Probability that UMD outperforms MKT over next year: {prob_out_single:.4f}")
print(f"Probability that UMD outperforms MKT over next 15 years: {prob_out_15y:.4f}")

Probability that UMD outperforms MKT over next year: 0.2780
Probability that UMD outperforms MKT over next 15 years: 0.0113


### 3.5.
Conceptually, why is there such a discrepancy between this probability for 1 period vs. 15 years?

What assumption about the log-returns are we making when we use this technique to estimate underperformance?

Since mean returns scale with time, and volatility scales with the square root of time, cumulative mean return trends higher than cumulative volatility, reducing the odds of "underperformance" as time grows. We are also assuming that log returns are normally distributed and independent.

### 3.6.

Using your previous answers, explain what is meant by time diversification.

Time diversification is the concept that holding investments over long periods reduces the probability of negative cumulative returns. Given the facts about how returns and volatility scale in my prevous answer, this means Sharpe ratio grows over time, too.

### 3.7.

Is the probability that `HML` and `UMD` both have negative cumulative returns over the next year higher or lower than the probability that `HML` and `MKT` both have negative cumulative returns over the next year?

Answer conceptually, but specifically. (No need to calculate the specific probabilities.)

HML and UMD both having negative cumulative returns is a LOWER probability than HML and MKT both having negative cumulative returns. Since we have observed HML and UMD to have little or negative correlations, while we have seen the opposite for HML and MKT, we are more likely to see opposite returns out oh HML and UMD than similar returns.

***